In [4]:
# --- Install dependencies ---
# !pip install -q transformers langchain accelerate bitsandbytes langchain_community

# --- 1. Load LLaMA 3 model locally via Hugging Face ---
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline # Import pipeline from the correct submodule
from langchain_community.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import SequentialChain
import torch

# Replace with your Hugging Face token (store securely in practice)
from google.colab import userdata
hf_token = userdata.get('llama2')
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Wrap with pipeline
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7
)

# Wrap pipeline in LangChain
llm = HuggingFacePipeline(pipeline=llama_pipeline)

# --- 2. Create PromptTemplates and LLMChains ---

# Explanation Chain
explanation_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain the concept of {topic} in simple terms for a high school student."
)
explanation_chain = LLMChain(llm=llm, prompt=explanation_prompt, output_key="explanation")

# Quiz Chain
quiz_prompt = PromptTemplate(
    input_variables=["topic", "explanation"],
    template="Based on the topic '{topic}' and the explanation below:\n\n{explanation}\n\nWrite a quiz question."
)
quiz_chain = LLMChain(llm=llm, prompt=quiz_prompt, output_key="quiz")

# Hint Chain
hint_prompt = PromptTemplate(
    input_variables=["quiz"],
    template="Provide a helpful hint to guide a student in answering the quiz question:\n\n{quiz}"
)
hint_chain = LLMChain(llm=llm, prompt=hint_prompt, output_key="hint")

# --- 3. Combine all into a SequentialChain ---
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[explanation_chain, quiz_chain, hint_chain],
    input_variables=["topic"],
    output_variables=["explanation", "quiz", "hint"],
    verbose=True
)

# --- 4. Run the chain ---
topic = "Neural Networks"
result = overall_chain.invoke({"topic": topic})

# --- 5. Print results ---
print("\n📘 Explanation:\n", result["explanation"])
print("\n❓ Quiz Question:\n", result["quiz"])
print("\n💡 Hint:\n", result["hint"])

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-4-1131311538.py:37: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llama_pipeline)
/tmp/ipython-input-4-1131311538.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  explanation_chain = LLMChain(llm=llm, prompt=explanation_prompt, output_key="explanation")




> Entering new SequentialChain chain...


KeyboardInterrupt: 

In [6]:
!pip install -q --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 32.9 MB/s eta 0:00:00
